LIBRARIES

In [1]:
#Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, preprocessing
from keras.preprocessing import image 
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array

DIRECTORIES

In [2]:
#Test to check if input folder images is detected
#print(os.listdir('input'))

In [3]:
#Directories
train_dir = 'input/train'
valid_dir = 'input/valid'
test_dir = 'input/test'

BIRD CLASSES

In [4]:
#Test to check if the 15 bird classes are detected
#print(os.listdir(train_dir))
#print(os.listdir(test_dir))
#print(os.listdir(valid_dir))

In [5]:
#Bird classes identification
bird_classes = os.listdir(train_dir)
#Test to check the names of the 15 bird classes
#print(str(len(bird_classes)),'The chosen 15 classes are ', bird_classes)

num_classes = len(bird_classes)
#Test to verify we have the correct amount of bird classes in each folder
#print(len(os.listdir(train_dir)))
#print(len(os.listdir(test_dir)))
#print(len(os.listdir(valid_dir)))

IMAGES

In [6]:
#Dimensions of image and input shapes
image_size = (224,224)
input_shape = (224, 224, 3)

#Batch size and number of epochs
batch_size = 32
epochs = 10

In [7]:
#Numpy arrays of the images of the training dataset
x_train=[]
for folder in os.listdir(train_dir):
    sub_path=train_dir+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,image_size)
        x_train.append(img_arr)

#Numpy arrays of the images of the testing dataset
x_test=[]
for folder in os.listdir(test_dir):
    sub_path=test_dir+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,image_size)
        x_test.append(img_arr)

#Numpy arrays of the images of the validation dataset
x_valid=[]
for folder in os.listdir(valid_dir):
    sub_path=valid_dir+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,image_size)
        x_valid.append(img_arr)

#Normalization of arrays
train_x = np.array(x_train)
test_x = np.array(x_test)
valid_x = np.array(x_valid)
train_x = train_x/255.0
test_x = test_x/255.0
valid_x = valid_x/255.0

In [8]:
#Rescale images
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)
valid_datagen = ImageDataGenerator(rescale=1/255)

In [9]:
#Read training images
train_data = train_datagen.flow_from_directory(train_dir, target_size = image_size, batch_size = batch_size)
test_data = test_datagen.flow_from_directory(test_dir, target_size = image_size, batch_size = batch_size)
valid_data = valid_datagen.flow_from_directory(valid_dir, target_size = image_size, batch_size = batch_size)

Found 2173 images belonging to 15 classes.
Found 75 images belonging to 15 classes.
Found 75 images belonging to 15 classes.


In [10]:
train_y = train_data.classes
test_y = test_data.classes
valid_y = valid_data.classes

In [11]:
train_data.class_indices

{'ABBOTTS BABBLER': 0,
 'BAIKAL TEAL': 1,
 'CACTUS WREN': 2,
 'DARK EYED JUNCO': 3,
 'EARED PITA': 4,
 'FAIRY BLUEBIRD': 5,
 'GAMBELS QUAIL': 6,
 'HAMMERKOP': 7,
 'IBERIAN MAGPIE': 8,
 'JABIRU': 9,
 'KAGU': 10,
 'LARK BUNTING': 11,
 'MAGPIE GOOSE': 12,
 'NICOBAR PIGEON': 13,
 'OCELLATED TURKEY': 14}

In [12]:
train_y.shape,test_y.shape,valid_y.shape

((2173,), (75,), (75,))

In [13]:
#Analyzing and previewing the training data
#list_bird_classes = list(train_generator.class_indices.keys())
#x = list(train_generator.classes)

#list_num_training_img = []

#for i in range (0,num_classes):
    #list_num_training_img.append(x.count(i))
    
#print('Number of training images per class varies between: ' + str(max(list_num_training_img)) 
      #+ ' and ' + str(min(list_num_training_img))

#Plotting bar chart of training images by bird class
#fig, ax = plt.subplots(figsize = (15, 4))
#sns.barplot(x = list_bird_classes,y = list_num_training_img)
#plt.xticks(rotation=90)
#plt.show()

CNN MODEL

In [14]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [15]:
#Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 186624)            0         
                                                                 
 dropout (Dropout)           (None, 186624)            0

In [16]:
#Compile model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
#Batch size and number of epochs

history = model.fit(train_data, batch_size = batch_size, epochs = epochs, validation_data = valid_data)

Epoch 1/10
63/68 [==========================>...] - ETA: 29s - loss: 2.3383 - accuracy: 0.2986

KeyboardInterrupt: 

In [ ]:
score = model.evaluate(test_data, batch_size = batch_size)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
#Save and export model (if needed)
#model.save_weights('CNN_model_saved.h5')

In [ ]:
# Accuracies
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
# Loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()

In [ ]:
#Classification matrix
preds = np.round(model.predict(test_data),0) 
print('rounded test_labels', preds)

classification_metrics = metrics.classification_report(test_data.classes, preds, target_names = bird_classes )
print(classification_metrics)

In [ ]:
#Confusion matrix
categorical_test_labels = pd.DataFrame(test_data.classes).idxmax(axis=1)
categorical_preds = pd.DataFrame(preds).idxmax(axis=1)
confusion_matrix= confusion_matrix(categorical_test_labels, categorical_preds)
#To get better visual of the confusion matrix:
def plot_confusion_matrix(cm, classes,
             normalize=False,
             title='Confusion matrix',
             cmap=plt.cm.Blues):
    #Add Normalization Option
    '''prints pretty confusion metric with normalization option '''
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    
#     print(cm)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

plot_confusion_matrix(confusion_matrix, ['butterflies', 'chickens', 'elephants', 'horses', 'spiders', 'squirells'],
                     normalize=True)

In [ ]:
#Prediction test on images
def read_image(file_path):
    print("[INFO] loading and preprocessing image...")  
    image = load_img(file_path, target_size=(224, 224))  
    image = img_to_array(image)  
    image = np.expand_dims(image, axis=0)
    image /= 255.  
    return image
def test_single_image(path):
    animals = ['butterflies', 'chickens', 'elephants', 'horses', 'spiders', 'squirells']
    images = read_image(path)
    time.sleep(.5)
    bt_prediction = vgg16.predict(images)  
    preds = model.predict_proba(bt_prediction)
    for idx, animal, x in zip(range(0,6), animals , preds[0]):
        print("ID: {}, Label: {} {}%".format(idx, animal, round(x*100,2) ))
    print('Final Decision:')
    time.sleep(.5)
    for x in range(3):
        print('.'*(x+1))
        time.sleep(.2)
    class_predicted = model.predict_classes(bt_prediction)
    class_dictionary = generator_top.class_indices  
    inv_map = {v: k for k, v in class_dictionary.items()}  
    print("ID: {}, Label: {}".format(class_predicted[0], inv_map[class_predicted[0]]))  
    return load_img(path)
path = 'data/test/butterears2.jpg'
test_single_image(path)
